In [1]:
do_mount=False
if do_mount:
  from google.colab import drive
  drive.mount('/content/gdrive')

In [2]:
# Parameters
do_save_activations = True
do_save_model = True

TRAIN_INPUT = 'twitgen_train_201906011956.csv'
VALID_INPUT = 'twitgen_valid_201906011956.csv'
TEST_INPUT = 'twitgen_test_201906011956.csv'
EMBEDDING_DIM = 200
MAXLEN = 50  # Maximum number of words per tweet that will be processed

In [3]:
basepath = '../data/'
glovefile = 'glove.twitter.27B.200d.txt.gz'
glovepath = basepath + glovefile

In [4]:
if do_mount:
  # Get the embedding initialization file
  !cp '$glovepath' .
  !gunzip $glovefile
  !ls -l

In [5]:
import tensorflow as tf
import pandas as pd
import os
import re
import keras
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
from keras.optimizers import Adam, Adagrad
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from datetime import datetime
import string

keras.__version__

Using TensorFlow backend.


'2.2.4'

In [6]:
# Read in the data
df_train = pd.read_csv(basepath+TRAIN_INPUT, index_col=['id','time'], parse_dates=['time'])
df_valid = pd.read_csv(basepath+VALID_INPUT, index_col=['id','time'], parse_dates=['time'])
df_test = pd.read_csv(basepath+TEST_INPUT, index_col=['id','time'], parse_dates=['time'])
df_train.head()

,,text,male
id,time,,
1083596943807393792,2019-05-27 23:27:08+00:00,"Ah, the Flat Earth gambit.\nWell played.",True
815783987784187904,2019-05-24 15:36:01+00:00,Aw ily,False
3458239641,2019-05-24 19:00:37+00:00,I hate being sick,False
1003729876250226688,2019-05-26 12:53:00+00:00,You still didn't' do shit tho. Slow down the...,True
2360143940,2019-05-28 03:50:46+00:00,Harriet Tubman may not be on the $20 bill... f...,False


In [7]:
# Maximum number of words per tweet in each data set
(df_train.text.str.split().apply(len).max(), 
 df_valid.text.str.split().apply(len).max(),
 df_test.text.str.split().apply(len).max())

(34, 30, 31)

In [8]:
# Text Normalization function

# Taken from 
# https://medium.com/@sabber/classifying-yelp-review-comments-using-lstm-and-word-embeddings-part-1-eb2275e4066b
# which was taken from https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings
# but this version no longer does stemming or stop word elmination

# This is for general text, not Twitter-specific.
# Maybe would get a better classifier if we used a Python transaltion of this:
# https://nlp.stanford.edu/projects/glove/preprocess-twitter.rb
# but that is arguably outside the scope of this project
# and my initial attempts to use Twitter-specific preprocessing have been unsuccessful


def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text


In [9]:
# Process the data for model input
def get_texts_and_labels(df):
  texts = df['text'].map(lambda x: clean_text(x)).tolist()
  texts = [t.split()[0:MAXLEN] for t in texts]
  labels = df['male'].tolist()
  return texts, labels

train_text, train_label = get_texts_and_labels(df_train)
valid_text, valid_label = get_texts_and_labels(df_valid)
test_text, test_label = get_texts_and_labels(df_test)

max([len(x) for x in train_text]), max([len(x) for x in valid_text]), max([len(x) for x in test_text])

(47, 42, 42)

In [10]:
# Fit tokenizer on training data
tok = Tokenizer()
tok.fit_on_texts(train_text)
vocab_size = len(tok.word_index) + 1

In [11]:
# Tokenize the data
def get_tokenized_texts(texts):
  encoded_docs = tok.texts_to_sequences(texts)
  padded_docs = pad_sequences(encoded_docs, maxlen=MAXLEN, padding='post')
  return padded_docs

docs_train = get_tokenized_texts(train_text)
docs_valid = get_tokenized_texts(valid_text)
docs_test = get_tokenized_texts(test_text)

print(type(docs_train), len(docs_train), len(docs_valid), len(docs_test))
docs_train[0][:10]

<class 'numpy.ndarray'> 34146 10914 10450


array([  956,     1,  4035,  1154, 13312,     8,    98,   732,     0,
           0], dtype=int32)

In [12]:
# Load the whole embedding into memory
embeddings_index = dict()
f = open(glovefile[:-3])
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [13]:
# Create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tok.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [14]:
# NERUAL NETWORK MODEL

# PARAMETERS

batchsize = 512

temporal_dropout = .2
random_dropout = .1
flat_dropout= .3

frozen_lr = 1e-3
frozen_decay = 1e-4
frozen_epochs = 12

unfrozen_lr = 3e-4
unfrozen_decay = 3e-5
unfrozen_epochs = 25
unfrozen_batchsize = batchsize


inputs = layers.Input((MAXLEN,), dtype="int32")

# EMBEDDING BLOCK
raw_embed = layers.Embedding(vocab_size, 
                           EMBEDDING_DIM, 
                           weights=[embedding_matrix], 
                           input_length=MAXLEN, 
                           trainable=False)(inputs)
embed_random_drop = layers.Dropout(rate=random_dropout)(raw_embed)
embed_time_drop = layers.Dropout(rate=temporal_dropout, 
                       noise_shape=(None, MAXLEN, 1))(embed_random_drop)

max_pooled = layers.GlobalMaxPooling1D()(embed_time_drop)
av_pooled = layers.GlobalAveragePooling1D()(embed_time_drop)
pooled = layers.merge.concatenate([max_pooled,av_pooled], name='pooled')
drop_pooled = layers.Dropout(rate=flat_dropout)(pooled)

pred = layers.Dense(1, activation='sigmoid')(drop_pooled)

# FINAL MODEL
model = Model(inputs=[inputs], outputs=pred)
model.compile(optimizer=Adam(frozen_lr, decay=frozen_decay), 
              loss='binary_crossentropy', metrics=['acc'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 200)      6026200     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 50, 200)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
dropout_2 (Dropout)  

In [15]:
model.layers

In [16]:
# Fit the frozen model

model.fit(docs_train, train_label, epochs=frozen_epochs, 
          validation_data=(docs_valid, valid_label), batch_size=batchsize)

Instructions for updating:
Use tf.cast instead.
Train on 34146 samples, validate on 10914 samples
Epoch 1/12
34146/34146 [==============================] - 2s 61us/step - loss: 0.7377 - acc: 0.5028 - val_loss: 0.6923 - val_acc: 0.5248
Epoch 2/12
34146/34146 [==============================] - 2s 48us/step - loss: 0.7123 - acc: 0.5166 - val_loss: 0.6864 - val_acc: 0.5451
Epoch 3/12
34146/34146 [==============================] - 2s 48us/step - loss: 0.7005 - acc: 0.5302 - val_loss: 0.6806 - val_acc: 0.5713
Epoch 4/12
34146/34146 [==============================] - 2s 49us/step - loss: 0.6943 - acc: 0.5433 - val_loss: 0.6775 - val_acc: 0.5839
Epoch 5/12
34146/34146 [==============================] - 2s 49us/step - loss: 0.6876 - acc: 0.5519 - val_loss: 0.6753 - val_acc: 0.5870
Epoch 6/12
34146/34146 [==============================] - 2s 49us/step - loss: 0.6837 - acc: 0.5562 - val_loss: 0.6744 - val_acc: 0.5826
Epoch 7/12
34146/34146 [==============================] - 2s 49us/step - loss: 0

In [17]:
[(l.name, l.trainable) for l in model.layers]

[('input_1', False),
 ('embedding_1', False),
 ('dropout_1', True),
 ('dropout_2', True),
 ('global_max_pooling1d_1', True),
 ('global_average_pooling1d_1', True),
 ('pooled', True),
 ('dropout_3', True),
 ('dense_1', True)]

In [18]:
# Unfreeze the embeddings
model.layers[1].trainable = True
model.compile(optimizer=Adam(unfrozen_lr, decay=unfrozen_decay), 
              loss='binary_crossentropy', metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 200)      6026200     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 50, 200)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 50, 200)      0           dropout_1[0][0]                  
__________________________________________________________________________________________________
global_max

In [19]:
# Fit the unfrozen model

model.fit(docs_train, train_label, epochs=unfrozen_epochs, 
          validation_data=(docs_valid, valid_label), batch_size=unfrozen_batchsize)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 34146 samples, validate on 10914 samples
Epoch 1/25
34146/34146 [==============================] - 7s 192us/step - loss: 0.6713 - acc: 0.5843 - val_loss: 0.6685 - val_acc: 0.6012
Epoch 2/25
34146/34146 [==============================] - 6s 169us/step - loss: 0.6696 - acc: 0.5871 - val_loss: 0.6684 - val_acc: 0.6017
Epoch 3/25
34146/34146 [==============================] - 6s 169us/step - loss: 0.6663 - acc: 0.5918 - val_loss: 0.6671 - val_acc: 0.5977
Epoch 4/25
34146/34146 [==============================] - 6s 169us/step - loss: 0.6651 - acc: 0.5952 - val_loss: 0.6664 - val_acc: 0.5990
Epoch 5/25
34146/34146 [==============================] - 6s 169us/step - loss: 0.6623 - acc: 0.5998 - val_loss: 0.6657 - val_acc: 0.6015
Epoch 6/25
34146/34146 [==============================] - 6s 169us/step - loss: 0.6605 - acc: 0.6032 - val_loss: 0.6650 - val_acc: 0.6023
Epoch 7/25
34146/34146 [====================

In [20]:
timestamp = datetime.now().strftime('%Y%m%d%H%M')

In [21]:
if do_save_activations:

  inp = model.input
  layer_name = 'pooled'
  activations = model.get_layer(layer_name).output
  get_activations = K.function([inp, K.learning_phase()], [activations])

  def get_inference_activations(docs):
    return(get_activations([docs, 0])[0])
  
  def save_activations(df, docs, split, timestamp):

    split_activations = get_inference_activations(docs)

    for i in range(split_activations.shape[1]):
      df['activ'+str(i)] = (split_activations[:,i]*1e6).astype(int)

    activ_file_name = basepath + 'activ_' + split + '_' + timestamp + '.csv'
    print(activ_file_name)
    df.drop(['text'],axis=1).to_csv(activ_file_name)
    
  save_activations(df_train, docs_train, 'train', timestamp)
  save_activations(df_valid, docs_valid, 'valid', timestamp)
  save_activations(df_test, docs_test, 'test', timestamp)
  

../data/activ_train_201907041941.csv
../data/activ_valid_201907041941.csv
../data/activ_test_201907041941.csv


In [22]:
if do_save_model:
    model_file_name = basepath + 'PoolModel_' + timestamp + '.h5'
    model.save(model_file_name)
    print(model_file_name)

../data/PoolModel_201907041941.h5
